In [ ]:
# Pancreatic Cancer Detection Notebook

In [2]:
import pandas as pd

# Data Loading and Preprocessing 

Ideas we should probably do for both datasets:
    - load data. remove any Null/NaN data, see if there are negative values and remove those if applicable.
    - if blank data cells, impute the data. scale the appropriate data columns


    - split the dataset into X and y subsets for passing into models
        - which really means also split into 80/20; 80% training(validation included) and 20% test

    - Note: we should do some data visualizations --> heatmaps? histograms of label distributions for proof
        - of good splits. mutual information??? (need to better understand if its useful)


# Model Creation

- we want to initialize all 4 models, probably default hyperparameters for most values.
    - it's 4 models per dataset, so we'll have 8 in total

# Training and Validation

- Train the models using 10-fold cross validation 
    - we can probably do this in 2 separate cross-validation loops. 
        - first one is for urinary dataset and we train/validate all 4 models (get their scores and loss? too)
        - repeat for second dataset

# Testing Models

- Test each of the models on the appropriate dataset and store values


# Significance Analysis of Results

- Do the McNemar test here for each of the models (8?) to compare

# Comparison Across Datasets

- take urinary dataset models and try to predict on other dataset's test samples and vice versa
    - draw conclusions on the accuracy percentage and based on this, we can determine if the urinary biomarkers
        - are generalizeable to use on other data for predicting pancreatic cancer

# ------------- NON URINARY DATA WORK --------

In [12]:
rawData1 = pd.read_csv("pancreaticCancerRNAseqData/GSE232860_allsamples.deseq.normalized.counts.csv")
#print(rawData1.shape)


#Slow loading, 48,553 rows
#rawData2 = pd.read_excel("pancreaticCancerRNAseqData/GSE245306_FKPM.xlsx")
#print(rawData2.shape)

#59050 rows
rawData3 = pd.read_csv("pancreaticCancerRNAseqData/tumor.counts.sub.tsv", sep='\t')
#print(rawData3)

             pat_43_T  pat_01_T  pat_02_T  pat_45_T  pat_03_T  pat_04_T  \
DDX11L1             0         0         0         0         0         0   
WASH7P              1         2        13         3         1         0   
MIR6859-1          15        15        23         1        22         2   
MIR1302-2HG         0         0         0         0         0         0   
MIR1302-2           0         0         0         0         0         0   
...               ...       ...       ...       ...       ...       ...   
MT-ND6           9218      2687      2761      3587      2062      5795   
MT-TE               9        11         1         4         0         5   
MT-CYB         146786     66226     34925     54750     49839    463102   
MT-TT              91        11        21        94        18        59   
MT-TP              27         5         5         1        14        37   

             pat_05_T  pat_06_T  pat_48_T  pat_07_T  ...  pat_56_T  pat_74_T  \
DDX11L1            